# TypeError: No conversion path for dtype: dtype('<U53')

In [1]:
import h5py, os, tqdm
import numpy as np

In [2]:
mesh_dir = 'datasets/ShapeNetSem/models-COLLADA/COLLADA_mini'
grasps_dir = 'datasets/grasps'
out_dir = 'datasets/output'

In [3]:
from scenecollisionnet.utils import (
    MeshLoader,
    ProcessKillingExecutor,
    process_mesh,
)

In [4]:
mesh_loader = MeshLoader(mesh_dir)

In [5]:
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
out_ds_file = os.path.join(out_dir, "object_info.hdf5")

In [6]:
os.listdir(grasps_dir)

['Stapler_b5fc0f0125873b3d4db629c0c07799a3_0.0015928690119586671.h5',
 'Helicopter_88a3ff85cc863b424312c0bbcc2e4932_0.0001384623265194279.h5',
 'Chair_66b29a0b56c2b69d25af37f812eb1801_0.00031492437374890846.h5',
 'Guitar_7eac9185f94e7be35fd682b25025f90d_0.0073912699214687535.h5',
 'Faucet_346a94616fa138bb85d4c34ee84c24a9_0.022706594945899414.h5',
 'Room_room15_0.0006049088409052.h5',
 'FloorLamp_94385beb66077dcd120c750dcecab484_0.001274700007905349.h5',
 'Chair_5eb6ad0960250dddffc127a39b3c57b1_0.0044484123743973895.h5',
 'Chair_7bdc0aaca74cca86593ebeeedbff73b_0.005179707932635725.h5',
 'FloorLamp_185e37856f55b2c4e9aa07397ea1cf8d_0.004798392206827627.h5',
 'Chair_ef13f960f08edac3b91b871e750ca615_0.008415915955730732.h5',
 'Curtain_b06c7a8e2cf23b7735836c728d324152_0.0011417229942700916.h5',
 'Plant_814ee1db103a369d2f9f1429cfcfb0ad_0.0002131430516292384.h5',
 'Printer_9024205a26da1d5c8e8e70bdbd6cb1df_0.014828043191572122.h5',
 'Guitar_c4d76ad65741ed1cdf16e78e07c53dc6_0.0001008605802347800

In [7]:
mesh_loader._map

{'1a4ec387ea6820345778775dfd5ca46a': 'datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a4ec387ea6820345778775dfd5ca46a.dae',
 '1a5e21a5633e1134ebd90840d80c9784': 'datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a5e21a5633e1134ebd90840d80c9784.dae',
 '1a6a67905880e4911a4d5e0a785b0e03': 'datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a6a67905880e4911a4d5e0a785b0e03.dae',
 '1a4daa4904bb4a0949684e7f0bb99f9c': 'datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a4daa4904bb4a0949684e7f0bb99f9c.dae',
 '1a3efcaaf8db9957a010c31b9816f48b': 'datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a3efcaaf8db9957a010c31b9816f48b.dae'}

In [8]:
inputs = []
for f in os.listdir(grasps_dir):
    mc, mk, ms = os.path.splitext(f)[0].split("_")
    #print(mk)
    try:
        in_path = mesh_loader.get_path(mk)
        #print(in_path)
    except ValueError:
        continue
    out_path = os.path.join(
        out_dir,
        mc,
        os.path.splitext(os.path.basename(in_path))[0] + ".obj",
    )
    inputs.append(
        (in_path, out_path, float(ms), os.path.join(grasps_dir, f))
    )

In [9]:
inputs

[('datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a4ec387ea6820345778775dfd5ca46a.dae',
  'datasets/output/Speaker/1a4ec387ea6820345778775dfd5ca46a.obj',
  0.004405836004468139,
  'datasets/grasps/Speaker_1a4ec387ea6820345778775dfd5ca46a_0.004405836004468139.h5'),
 ('datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a6a67905880e4911a4d5e0a785b0e03.dae',
  'datasets/output/Sink/1a6a67905880e4911a4d5e0a785b0e03.obj',
  0.0011058383813692342,
  'datasets/grasps/Sink_1a6a67905880e4911a4d5e0a785b0e03_0.0011058383813692342.h5'),
 ('datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a5e21a5633e1134ebd90840d80c9784.dae',
  'datasets/output/Lamp/1a5e21a5633e1134ebd90840d80c9784.obj',
  0.02338623474048206,
  'datasets/grasps/Lamp_1a5e21a5633e1134ebd90840d80c9784_0.02338623474048206.h5'),
 ('datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a3efcaaf8db9957a010c31b9816f48b.dae',
  'datasets/output/Laptop/1a3efcaaf8db9957a010c31b9816f48b.obj',
  0.011789003201190232,
  'datasets/grasps/Laptop_1a

In [64]:
inputs[0][0]

'datasets/ShapeNetSem/models-COLLADA/COLLADA_mini/1a4ec387ea6820345778775dfd5ca46a.dae'

In [66]:
import trimesh
mesh = trimesh.load(inputs[0][0], force="mesh", skip_materials=True)
cat = (
    ""
    if os.path.basename(os.path.dirname(inputs[0][1])) == "meshes"
    else os.path.basename(os.path.dirname(inputs[0][1]))
)

In [69]:
print(mesh)
print(cat)

<trimesh.Trimesh(vertices.shape=(6170, 3), faces.shape=(6274, 3))>
Speaker


In [72]:
scale = inputs[0][2]
m_scale = "{}_{}.obj".format(
    os.path.splitext(os.path.basename(out_path))[0], scale
)
m_info = {
    "path": os.path.join(cat, m_scale),
    "scale": scale,
    "category": cat,
}
print(m_scale)
print(m_info)

1a4daa4904bb4a0949684e7f0bb99f9c_0.004405836004468139.obj
{'path': 'Speaker/1a4daa4904bb4a0949684e7f0bb99f9c_0.004405836004468139.obj', 'scale': 0.004405836004468139, 'category': 'Speaker'}


In [10]:
def process_mesh_timeout(*args, **kwargs):
    pass

In [119]:
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
out_ds_file = os.path.join(out_dir, "object_info.hdf5")

executor = ProcessKillingExecutor(max_workers=8)
generator = executor.map(
    process_mesh, # scenecollisionnet utils.py
    inputs, # in_path, out_path, scale, grasps
    timeout=120,
    callback_timeout=process_mesh_timeout,
)

with h5py.File(out_ds_file, "a") as f:
    if "meshes" not in f:
        f.create_group("meshes")

    categories = {}
    if "categories" not in f:
        f.create_group("categories")

    for mesh_info in generator:
        if mesh_info is not None:
            mk, minfo = mesh_info
            print(mk)
            print(m_info)
            if mk not in f["meshes"]:
                f["meshes"].create_group(mk)
            for key in minfo:
                if key in f["meshes"][mk]:
                    del f["meshes"][mk][key]
                if isinstance(minfo[key], str):
                    f["meshes"][mk][key] = [minfo[key].encode('ascii')]
                    print(f["meshes"][mk][key])
                else:
                    f["meshes"][mk][key] = [minfo[key]]

            if minfo["category"] not in categories:
                categories[minfo["category"]] = [mk.encode('ascii')]
            elif mk not in categories[minfo["category"]]:
                categories[minfo["category"]].append(mk.encode('ascii'))
    
    print(categories)
    for c in categories:
        if c in f["categories"]:
            del f["categories"][c]
        print(c)
        print(categories[c])
        f["categories"][c] = categories[c]
        print(f["categories"][c])

1a4ec387ea6820345778775dfd5ca46a_0.004405836004468139
{'path': 'Speaker/1a4daa4904bb4a0949684e7f0bb99f9c_0.004405836004468139.obj', 'scale': 0.004405836004468139, 'category': 'Speaker'}
<HDF5 dataset "path": shape (1,), type "|S65">
<HDF5 dataset "category": shape (1,), type "|S7">
1a6a67905880e4911a4d5e0a785b0e03_0.0011058383813692342
{'path': 'Speaker/1a4daa4904bb4a0949684e7f0bb99f9c_0.004405836004468139.obj', 'scale': 0.004405836004468139, 'category': 'Speaker'}
<HDF5 dataset "path": shape (1,), type "|S63">
<HDF5 dataset "category": shape (1,), type "|S4">
1a5e21a5633e1134ebd90840d80c9784_0.02338623474048206
{'path': 'Speaker/1a4daa4904bb4a0949684e7f0bb99f9c_0.004405836004468139.obj', 'scale': 0.004405836004468139, 'category': 'Speaker'}
<HDF5 dataset "path": shape (1,), type "|S61">
<HDF5 dataset "category": shape (1,), type "|S4">
1a3efcaaf8db9957a010c31b9816f48b_0.011789003201190232
{'path': 'Speaker/1a4daa4904bb4a0949684e7f0bb99f9c_0.004405836004468139.obj', 'scale': 0.00440583

In [120]:
categories

{'Speaker': [b'1a4ec387ea6820345778775dfd5ca46a_0.004405836004468139'],
 'Sink': [b'1a6a67905880e4911a4d5e0a785b0e03_0.0011058383813692342'],
 'Lamp': [b'1a5e21a5633e1134ebd90840d80c9784_0.02338623474048206'],
 'Laptop': [b'1a3efcaaf8db9957a010c31b9816f48b_0.011789003201190232'],
 'Stapler': [b'1a4daa4904bb4a0949684e7f0bb99f9c_0.00040353195213810913']}

In [123]:
with h5py.File(out_ds_file, "a") as f:
    mk = list(f["meshes"].keys())
    print(type(f["meshes"][mk[0]]['path']))
    print(f["meshes"][mk[0]]['path'][0].decode())
    print(type(f["meshes"][mk[0]]['path'][0].decode()))

<class 'h5py._hl.dataset.Dataset'>
Laptop/1a3efcaaf8db9957a010c31b9816f48b_0.011789003201190232.obj
<class 'str'>


### TypeError: Can't iterate over a scalar dataset

h5py 파일에 데이터를 저장할 때 list가 아닌 scalar값으로 저장한 후 list로 변환을 시도할 때 에러가 발생함
이는 'TypeError: No conversion path for dtype: dtype('<U53')' 에러를 해결하기 위해 리스트에서 원소만 꺼내서 넣어서 발생했던 문제

In [20]:
with h5py.File(out_ds_file, "a") as f:
    print(f["categories"])
    c = 'Speaker'
    if c in f["categories"]:
        del f["categories"][c]
    f["categories"][c] = [1] # scalar가 아닌 list로 저장해야 shape이 생기고 리스트로 변환 시 에러가 발생하지 않음
    keys = list(f["categories"].keys())
    print(keys)
    print(f["categories"][c])
    list(f["categories"][c])
    
    c = 'Sink'
    if c in f["categories"]:
        del f["categories"][c]
    f["categories"][c] = 1 # scalar가 아닌 list로 저장해야 shape이 생기고 리스트로 변환 시 에러가 발생하지 않음
    keys = list(f["categories"].keys())
    print(keys)
    print(f["categories"][c])
    list(f["categories"][c])

<HDF5 group "/categories" (5 members)>
['Lamp', 'Laptop', 'Sink', 'Speaker', 'Stapler']
<HDF5 dataset "Speaker": shape (1,), type "<i8">
['Lamp', 'Laptop', 'Sink', 'Speaker', 'Stapler']
<HDF5 dataset "Sink": shape (), type "<i8">


TypeError: Can't iterate over a scalar dataset

### TypeError: No conversion path for dtype: dtype('<U53')

h5py 파일에 문자열을 저장 시 utf8 타입으로 인코딩하지 않으면 에러가 발생

In [26]:
with h5py.File(out_ds_file, "a") as f:
    print(f["categories"])
    c = 'Speaker'
    if c in f["categories"]:
        del f["categories"][c]
    f["categories"][c] = [str(1).encode('utf8')] 
    keys = list(f["categories"].keys())
    print(keys)
    print(f["categories"][c])
    list(f["categories"][c])
    
    c = 'Sink'
    if c in f["categories"]:
        del f["categories"][c]
    f["categories"][c] = [str(1)] # 
    keys = list(f["categories"].keys())
    print(keys)
    print(f["categories"][c])
    list(f["categories"][c])

<HDF5 group "/categories" (4 members)>
['Lamp', 'Laptop', 'Speaker', 'Stapler']
<HDF5 dataset "Speaker": shape (1,), type "|S1">


TypeError: No conversion path for dtype: dtype('<U1')

### 저장된 hdf5 파일 확인

In [1]:
import h5py, os
import numpy as np

obj_info = h5py.File(
    os.path.join('datasets/shapenet', "object_info.hdf5"), "r" 
)
mesh_info = obj_info["meshes"]
categories = obj_info["categories"]

In [2]:
print(obj_info)
print(mesh_info)

<HDF5 file "object_info.hdf5" (mode r)>
<HDF5 group "/meshes" (6488 members)>


In [3]:
print(categories)
print(list(categories.keys()))

<HDF5 group "/categories" (256 members)>
['1Shelves', '2Shelves', '3Shelves', '4Shelves', '5Shelves', '6Shelves', '7Shelves', 'AABattery', 'AccentChair', 'AccentTable', 'Airplane', 'Armoire', 'Backpack', 'Bag', 'Ball', 'Banana', 'Barstool', 'Basket', 'Bathtub', 'Battery', 'BeanBag', 'Bear', 'Bed', 'BeerBottle', 'Bench', 'Bird', 'Board', 'Book', 'Bookcase', 'Books', 'Bottle', 'Bowl', 'BreadSlice', 'Bucket', 'Cabinet', 'Cabling', 'Cage', 'Cake', 'Calculator', 'Camera', 'CanOpener', 'Candle', 'Canister', 'CanopyBed', 'Cap', 'Car', 'Carrot', 'Cassette', 'Cat', 'CeilingFan', 'CeilingLamp', 'CellPhone', 'CerealBox', 'Chair', 'Chaise', 'ChestOfDrawers', 'ChildBed', 'Chocolate', 'Coaster', 'CoatRack', 'CoffeeTable', 'Coin', 'Computer', 'ComputerMouse', 'Controller', 'Cookie', 'Copier', 'Couch', 'Counter', 'Courtyard', 'Cow', 'Cradle', 'Credenza', 'Cup', 'CupCake', 'CurioCabinet', 'Curtain', 'DecorativeAccessory', 'Desk', 'DeskLamp', 'Desktop', 'DiningTable', 'DiscCase', 'Dog', 'Donkey', 'Donut

In [4]:
cat = np.random.choice(list(categories.keys()))
print(cat)
print(categories[cat])
print(categories[cat][()])
print(categories[cat][()][0].decode())

Couch
<HDF5 dataset "Couch": shape (297,), type "|O">
[b'4ef1c8ad9c72263cdc6d207f01b16dd0_4.217449016991198e-06'
 b'ed4ac116e03ebb8d663191fd557d3a61_0.00042434707238079477'
 b'ee4ec19df3a6b81317fd40ad196436c_0.0015809289189816955'
 b'dccb04ee585d9d10cf065d4b58dc3084_0.0031744400792793185'
 b'ad5ef1b493028c0bd810b14a81e12eca_0.002453548497336451'
 b'31b5cb5dfaa253b3df85db41e3677c28_0.0014769275073730033'
 b'31b5cb5dfaa253b3df85db41e3677c28_0.000874918924912063'
 b'cdb7efad9942e4f9695ff9c22a7938c6_0.0016528567009595045'
 b'b38978ed5bb13266f2b67ae827b02632_0.0015446392309738724'
 b'e49c0df0a42bdbecc4b4c7225ff8487e_0.002499821521171607'
 b'3415f252bd71495649920492438878e5_0.00035118987370511896'
 b'151b2df36e08a13fafeeb1a322c90696_1.701306896759987e-05'
 b'b55d24ed786a8279ad2d3ca7f660ddd_0.0041860999107884295'
 b'9cff96bae9963ceab3c971099efd7fb9_0.003153862775189784'
 b'8efa91e2f3e2eaf7bdc82a7932cd806_0.0026942594712784603'
 b'4ed802a4aa4b8a86b161f36d4e309050_0.0017522683193317492'
 b'21b2

In [11]:
obj = np.random.choice(list(categories[cat]))
print(mesh_info[obj])
print(mesh_info[obj]['path'])
print(mesh_info[obj]['path'][()])

<HDF5 group "/meshes/886833189fd5d75864d18951dba4fc84_0.015619387405166914" (6 members)>
<HDF5 dataset "path": shape (), type "|O">
Pan/886833189fd5d75864d18951dba4fc84_0.015619387405166914.obj


In [12]:
from scenecollisionnet.datasets import BenchmarkSceneCollisionDataset
from autolab_core import BinaryClassificationResult, YamlConfig, keyboard_input
from torch.utils.data import DataLoader
import os.path as osp

cfg = 'cfg/benchmark_scenecollisionnet.yaml'
config = YamlConfig(cfg)
out = osp.join(config["output"], config["model"]["name"])

In [13]:
model_path = osp.join(config["model"]["path"], config["model"]["name"])
train_cfg = YamlConfig(osp.join(model_path, "train.yaml"))
kwargs = {
    "num_workers": 0
    if config["vis"]
    else (
        config["num_workers"]
        if "num_workers" in config
        else os.cpu_count()
    ),
    "pin_memory": True,
    "worker_init_fn": lambda _: np.random.seed(),
}

In [14]:
for k, v in config["dataset"].items():
    train_cfg["dataset"][k] = v
test_set = BenchmarkSceneCollisionDataset(
    **train_cfg["dataset"],
    **train_cfg["camera"],
    bounds=train_cfg["model"]["bounds"],
    vis=config["vis"]
)
test_loader = DataLoader(test_set, batch_size=None, **kwargs)

In [15]:
data_loader = iter(test_loader)
iterations = 10

In [16]:
for i in range(iterations):
    print(i)
    data = next(data_loader)

0


/root/share/pkgs/SceneCollisionNet/scenecollisionnet/scene.py:89: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if "grasps" in info:


1
2
3
4
5
6
7
8
9


In [9]:
test = list(np.zeros(8000))
len(test[:4000])

4000

In [14]:
len(test[4000:])

4000